In [1]:
import cv2

import numpy as np

import tensorflow as tf
from tensorflow import keras
from mtcnn.mtcnn import MTCNN

In [2]:
model = tf.keras.models.load_model("final_model.h5")

In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [4]:
# dictionary which assigns each label an emotion (alphabetical order)
emotion_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Sad", 5: "Surprise", 6: 'Neutral'}

In [5]:
img_size = 224

cv2.ocl.setUseOpenCL(False) # prevents openCL usage and unnecessary logging messages
detector = MTCNN()

In [6]:
# start the webcam feed
cap = cv2.VideoCapture(0)
black = np.zeros((50,50))

In [7]:
while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    if not ret:
        break
	# detect faces in the image
    results = detector.detect_faces(frame)
	# extract the bounding box from the first face
    if len(results) == 1: #len(results)==1 if there is a face detected. len ==0 if no face is detected
        try:
            x1, y1, width, height = results[0]['box']
            x2, y2 = x1 + width, y1 + height
        	# extract the face
            face = frame[y1:y2, x1:x2]
            #Draw a rectangle around the face
            cv2.rectangle(frame, (x1, y1), (x1 + width, y1 + height), (255, 0, 0), 2)
            # resize pixels to the model size
            cropped_img = cv2.resize(face, (img_size,img_size)) 
            cropped_img_expanded = np.expand_dims(cropped_img, axis=0)
            cropped_img_float = cropped_img_expanded.astype(float)
            prediction = model.predict(cropped_img_float)
            print(prediction)
            maxindex = int(np.argmax(prediction))
            cv2.putText(frame, emotion_dict[maxindex], (x1 + 20, y1 - 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        except:
            pass
    
    cv2.imshow('Video',frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 1s 870ms/step
[[0.0315826  0.00210895 0.09434202 0.03475061 0.02087524 0.72446567
  0.09187494]]
1/1 [==============================] - 0s 41ms/step
[[0.05713433 0.00970023 0.48048067 0.02257998 0.0245115  0.3044262
  0.10116714]]
1/1 [==============================] - 0s 41ms/step
[[0.0331111  0.00133375 0.62102437 0.00916386 0.00372539 0.12215605
  0.20948555]]
1/1 [==============================] - 0s 53ms/step
[[0.150374   0.00544838 0.40948868 0.05854571 0.0039356  0.05640899
  0.31579873]]
1/1 [==============================] - 0s 45ms/step
[[0.00891478 0.00114455 0.22815046 0.08728389 0.00407958 0.06711999
  0.6033068 ]]
1/1 [==============================] - 0s 146ms/step
[[1.5285749e-02 7.6586218e-04 7.8277308e-01 8.4649324e-03 3.3374624e-03
  3.3284910e-02 1.5608792e-01]]
1/1 [==============================] - 0s 43ms/step
[[0.05960557 0.00946091 0.36377424 0.00952578 0.02374823 0.45756996
  0.07631531]]
1/1 [============================

C:\Users\moez\anaconda3\envs\base2\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
C:\Users\moez\anaconda3\envs\base2\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
C:\Users\moez\anaconda3\envs\base2\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
C:\Users\moez\anaconda3\envs\base2\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
C:\Users\moez\anaconda3\envs\base2\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
C:\Users\moez\anaconda3\envs\base2\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':


KeyboardInterrupt: 